In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score, cohen_kappa_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/adwiz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adwiz/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/adwiz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ModuleNotFoundError: No module named 'gensim'

In [ ]:
df = pd.read_json('datasets/dataset.json')
df['labels'] = df.hasBadWords.apply(lambda x: "hasBadWords.True" if x == True else "hasBadWords.False")
# data['labels'] = df.hasBadWords.apply(lambda x: 1 if x == True else 0)
df.drop(['violation', 'hasBadWords'], axis=1, inplace=True)
# df['text'] = df['text'].apply(lambda x: str(x).split(' '))
df['text'] = df['text'].apply(lambda x: str(x))
# df.rename(columns={"text": "texts"}, inplace=True)
df['labels'] = df['labels'].astype('category')
print('dimension: ', df.shape)
df.head()

In [ ]:
df['labels'].value_counts()

In [ ]:
import re
regex = re.compile('[\w|\d]+') #('[A-Za-z]+')

def words_only(text, regex=regex):
    return ' '.join(regex.findall(text))

df.text = df.text.str.lower()
df.text = df.text.apply(words_only)

In [ ]:
df.head()

In [ ]:
df.text.iloc[0]

In [ ]:
from nltk.corpus import stopwords
mystopwords = stopwords.words('english') + ['I', 'div' , 'p', 'center', 'tr', 'com',  '</tr>', 'The', 'http',
                                           'font', 'br', 'href', 'clips4sale', 'www', 'align', 'td', 
                                            'img', 'src', 'class', 'span', '0', 'nbsp', 'b', 'imagecdn',
                                           '_blank', 'color', 'target', 'width', 'https://', 'jpg', 'bottom_html',
                                           'bottom_text', 'style', 'text', 'link', 'https', 'images', 'size',
                                           'border', 'height', 'true ', 'return true', 'wmv', 'sans', 'window status',
                                           'arial', 'helvetica']
def remove_stopwords(text, mystopwords=mystopwords):
    try:
        return ' '.join([token for token in text.split() if not token in mystopwords])
    except:
        return ''

df.text = df.text.apply(remove_stopwords)
df.text

In [2]:
%%time
# Unworked shit
from pymystem3 import Mystem

m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return ''.join(m.lemmatize(text)).strip()
    except:
        return ' '

##### Alternative way

# import nltk

# w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
# lemmatizer = nltk.stem.WordNetLemmatizer()

# def lemmatize(text):
#     return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

df.text = df.text.apply(lemmatize)

ModuleNotFoundError: No module named 'pymystem3'

In [4]:
!pip install gensim


  Using cached gensim-4.3.0-cp310-cp310-macosx_10_9_x86_64.whl (24.0 MB)
  Using cached FuzzyTM-2.0.5-py3-none-any.whl (29 kB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Using cached pyFUME-0.2.25-py3-none-any.whl (67 kB)
  Using cached fst_pso-1.8.1-py3-none-any.whl
  Using cached miniful-0.0.6-py3-none-any.whl


In [ ]:
df.head()

In [ ]:
# mystoplemmas = ['font','br','href', 'clips4sale', 'www', 'td', 'align', 'class', 'span']
# def remove_stoplemmas(text, mystoplemmas=mystoplemmas):
#     try:
#         return ' '.join([token for token in text.split() if not token in mystoplemmas])
#     except:
#         return ''

In [ ]:
df['labels'] = df['labels'].astype('category')
print('dimension: ', df.shape)
df.head()

In [ ]:
label_mapping = df['labels'].cat.categories
df['labels'] = df['labels'].cat.codes
X = df['text']
y = df['labels']

In [ ]:
test_size = 0.2
random_state = 1234

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state, stratify=y)

In [ ]:
from nltk import FreqDist

lemmata = []
for index, row in df.iterrows():
    lemmata  += row['text'].split()
fd = FreqDist(lemmata)
for i in fd.most_common(10):
    print(i)

In [ ]:
from wordcloud import *
word_freq = [i for i in fd.most_common(100)]
wd = WordCloud(background_color='white')
wd.generate_from_frequencies(dict(word_freq))
plt.figure()
plt.imshow(wd, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
tokens_by_topic = []
for topic in df['labels'].unique():
    tokens = []
    sample = df[df['labels'] == topic]
    for i in range(len(sample)):
        tokens += sample.text.iloc[i].split()
    tokens_by_topic.append(tokens)

In [ ]:
df['labels'].unique()

In [ ]:
event_id = 1
df['labels'].unique()[event_id]

In [ ]:
%%time
import nltk
from nltk import *
N_best = 100 # число извлекаемых биграмм

bigram_mesures = nltk.collocations.BigramAssocMeasures() # класс для мер ассоциации биграмм
finder = BigramCollocationFinder.from_words(tokens_by_topic[event_id]) # класс для хранения и извлечение биграм
finder.apply_freq_filter(3) # избавимся от биграм, которые встречаются реже 3 раз
raw_freq_ranking = [' '.join(i) for i in finder.nbest(bigram_mesures.raw_freq, N_best)] # выбираем топ 10 биграм по частоте
tscore_ranking = [' '.join(i) for i in finder.nbest(bigram_mesures.student_t, N_best)] # выбираем топ 10 биграм по категориям
pmi_ranking = [' '.join(i) for i in finder.nbest(bigram_mesures.pmi, N_best)]
llr_ranking = [' '.join(i) for i in finder.nbest(bigram_mesures.likelihood_ratio, N_best)]
chi2_ranking = [' '.join(i) for i in finder.nbest(bigram_mesures.chi_sq, N_best)]

In [ ]:
rankings = pd.DataFrame({'chi2': chi2_ranking, 'llr': llr_ranking, 't-score': tscore_ranking, 'pmi': pmi_ranking, 'raw': raw_freq_ranking})
rankings = rankings[['raw', 'pmi', 't-score', 'chi2', 'llr']]
rankings.head(10)

In [ ]:
from scipy.stats import spearmanr
import seaborn as sns
%matplotlib inline

corr = spearmanr(rankings).correlation
sns.heatmap(corr, annot=True, xticklabels = list(rankings), yticklabels = list(rankings));

In [ ]:
from sklearn.metrics import get_scorer_names
get_scorer_names()

In [ ]:
# lemmatizer + tokenizer (+ stemming) class
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        # we define (but not use) a stemming method, uncomment the last line in __call__ to get stemming tooo
        self.stemmer = nltk.stem.SnowballStemmer('english') 
    def __call__(self, doc):
        # pattern for numbers | words of length=2 | punctuations | words of length=1
        pattern = re.compile(r'[0-9]+|\b[\w]{2,2}\b|[%.,_`!"&?\')({~@;:#}+-]+|\b[\w]{1,1}\b')
        # tokenize document
        doc_tok = word_tokenize(doc)
        #filter out patterns from words
        doc_tok = [x for x in doc_tok if x not in stop_words1]
        doc_tok = [pattern.sub('', x) for x in doc_tok]
        # get rid of anything with length=1
        doc_tok = [x for x in doc_tok if len(x) > 1]
        # position tagging
        doc_tagged = nltk.pos_tag(doc_tok)
        # selecting nouns and adjectives
        doc_tagged = [(t[0], t[1]) for t in doc_tagged if t[1] in defTags]
        # preparing lemmatization
        doc = [(t[0], penn_to_wn(t[1])) for t in doc_tagged]
        # lemmatization
        doc = [self.wnl.lemmatize(t[0], t[1]) for t in doc]
        # uncomment if you want stemming as well
        #doc = [self.stemmer.stem(x) for x in doc]
        return doc

In [ ]:
vec_tdidf = TfidfVectorizer(ngram_range=(1,1), analyzer='word', #stop_words=stop_words1, 
                                               norm='l2', tokenizer=LemmaTokenizer())

In [ ]:
clf = XGBClassifier(random_state=42, seed=2, colsample_bytree=0.6, subsample=0.7)

In [ ]:
# label_mapping = data['labels'].cat.categories
# data['labels'] = data['labels'].cat.codes
# X = data['texts']
# y = data['labels']

In [ ]:
test_size = 0.2
random_state = 1234

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state, stratify=y)

# val_size = 0.1
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train, y_train, test_size=val_size, random_state=random_state, stratify=y_train)